In [7]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [8]:
sys.path.append(r"../LUCinSA_helpers")
from RF1_CreateModel import *

there are 4151 pts in the full data set
there are 3331 pts in the training set
there are 3218 sample points after removing those without clear class
Out-of-bag score estimate: 0.623
Mean accuracy score: 0.619
Confusion Matrix: predicted   1   3   7  12  13  17   31  33  34  35  ...  53  55  60  70  80  \
observed                                            ...                       
1          16  10   0   0   1   0    1   1   1   3  ...   0   0   0   0   0   
3           5  31   0   0   0   0    2   0   0   0  ...   0   2   0   0   0   
7           0   1  29   2   0   2    1   0   0   0  ...   0   0   0   1   1   
12          0   0   0  11   5   0    4   2   0   5  ...   0   3   0   0   0   
13          0   0   1   4   9   0    5   0   0   3  ...   0   1   0   1   0   
17          0   0   1   4   0   1    1   0   0   3  ...   0   0   0   0   0   
31          0   1   0   0   0   0  251   0   0   2  ...   0   0   0   0   0   
32          0   0   0   0   0   0    0   1   2   3  ...   0   

In [ ]:
in_dir = 'D:/NasaProject/Paraguay/ClassificationModels'
df_in = os.path.join(in_dir, 'RFdf.csv')
out_dir = 'D:/NasaProject/Paraguay/ClassificationModels/RF'

In [ ]:
## Multi-class model with all 22 classes

In [ ]:
RFmod = MulticlassRF(df_in,out_dir,'All','Permutation',29)

In [6]:
LUT=pd.read_csv('../Class_LUT.csv')
print(LUT)

                 USE_NAME   Class1  LC5  LC22  LC3  LC4  LC2  \
0              NoVeg_Bare    NoVeg   10     1    0    0    0   
1             NoVeg_Built    NoVeg   10     3    0    0    0   
2             NoVeg_Water    NoVeg   10     7    0    0    0   
3               Grassland   LowVeg   20    99    0    0    0   
4       Grassland-Natural   LowVeg   20    12    0    0    0   
5       Grassland-Planted   LowVeg   20    13    0    0    0   
6   Grassland-Natural-wet   LowVeg   20    17    0    0    0   
7               Crops-low   LowVeg   20    99    1    1    1   
8          Crops-Soybeans   LowVeg   20    31    1    1    1   
9             Crops-Beans   LowVeg   20    32    1    1    1   
10             Crops-Corn   LowVeg   20    33    1    1    1   
11         Crops-Mandioca   LowVeg   20    34    1    1    1   
12              Crops-mix   LowVeg   20    35    1    1    1   
13          TreePlant-new   LowVeg   20    36    1    1    1   
14             Crops-Rice   LowVeg   20 